In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = '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',
    model="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on the moon was..")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/sales-rep-nike-net-na/job/R-73031")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






















Sales Rep Nike.net NA










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Selec

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:
    {page_data}
    INSTRUCTION:
    The scraped text is from a career page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys:
    'role','experience','skills' and 'description'.
    Only return the valid JSON.
    ###Valid JSON(No PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
type(res.content)

```json
{
  "role": "Sales Rep Nike.net NA",
  "experience": "At least 2 years of experience that includes field sales, retail buying, merchandising, sales business planning or sales business analysis.",
  "skills": [
    "Strong communication skills – both written and verbal",
    "Strong organizational skills, the ability to multi task and prioritize",
    "Must have a self-starter approach and be able to work independently and autonomously",
    "Ability to work well with ambiguity and possess a creative problem-solving mentality",
    "Ability to navigate cross functional environment, be deadline drive and have a strong attention to detail.",
    "Retail acumen: Understanding of retail math, gross margin"
  ],
  "description": "We are looking for a digital first sales rep to position Nike as the leading brand within their assigned accounts by driving profitable and sustainable growth and market share. You will accomplish this by aligning with Nike’s Win Now strategy across North Am

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Sales Rep Nike.net NA',
 'experience': 'At least 2 years of experience that includes field sales, retail buying, merchandising, sales business planning or sales business analysis.',
 'skills': ['Strong communication skills – both written and verbal',
  'Strong organizational skills, the ability to multi task and prioritize',
  'Must have a self-starter approach and be able to work independently and autonomously',
  'Ability to work well with ambiguity and possess a creative problem-solving mentality',
  'Ability to navigate cross functional environment, be deadline drive and have a strong attention to detail.',
  'Retail acumen: Understanding of retail math, gross margin'],
 'description': 'We are looking for a digital first sales rep to position Nike as the leading brand within their assigned accounts by driving profitable and sustainable growth and market share. You will accomplish this by aligning with Nike’s Win Now strategy across North America. In this role, you will ma

In [9]:
type(json_res)

dict

In [10]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [18]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                   metadatas={"links": row["Links"]},
                   ids=[str(uuid.uuid4())])

In [23]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [22]:
job = json_res
job['skills']

['Strong communication skills – both written and verbal',
 'Strong organizational skills, the ability to multi task and prioritize',
 'Must have a self-starter approach and be able to work independently and autonomously',
 'Ability to work well with ambiguity and possess a creative problem-solving mentality',
 'Ability to navigate cross functional environment, be deadline drive and have a strong attention to detail.',
 'Retail acumen: Understanding of retail math, gross margin']

In [24]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Sales Rep Productivity with AtliQ's Automated Solutions

Dear Hiring Manager,

I came across the job description for a Sales Rep Nike.net NA and was impressed by the role's focus on driving profitable and sustainable growth through digital channels. As a Business Development Executive at AtliQ, I believe our company can help enhance the productivity and efficiency of your sales representatives.

At AtliQ, we specialize in developing tailored solutions that facilitate the seamless integration of business processes through automated tools. Our expertise in AI and software consulting can help your sales team streamline tasks, prioritize activities, and make data-driven decisions. With our solutions, your sales representatives can focus on high-value activities like building relationships, identifying new opportunities, and driving revenue growth.

Our portfolio showcases our capabilities in developing innovative solutions for various industries. For instance, our work i